In [0]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 2s (231 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 134443 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The 

EasyProcessError: ignored

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [0]:
import gym
env = gym.make("FrozenLake-v0")

import torch
import torch.nn as n
import torch.optim as op
import torch.nn.functional as f
import numpy as np

In [0]:
actiondim=env.action_space.n

In [0]:
obsdim=env.observation_space.n

In [0]:
from collections import namedtuple

In [0]:
episode=namedtuple("episode",field_names=["reward",'steps'])
episodestep=namedtuple("episodestep",field_names=['actions','observations'])

In [0]:
env.action_space.n

4

In [0]:
observationspace=gym.spaces.Box(0.0,1.0,shape=(env.observation_space.n,),dtype=np.float32)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
observationspace.low

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [0]:
class discreteonehotwrapper(gym.ObservationWrapper):
  def __init__(self,env):
    super().__init__(env)
    self.observationspace=gym.spaces.Box(0.0,1.0,shape=(env.observation_space.n,),dtype=np.float32)
  
  def observation(self,x):
    print(x)
    res=np.copy(self.observationspace.low)
    res[x]=1.0
    return res

In [0]:
batchsize=100

In [0]:
episode=namedtuple("episode",field_names=['reward','steps'])
episodestep=namedtuple("episodestep",field_names=['observation','actions'])

In [0]:
class model(n.Module):
  def __init__(self,indim,middim,outdim):
    super().__init__()
    self.fc1=n.Linear(indim,middim[0])
    self.fc2=n.Linear(middim[0],middim[1])
    self.fc3=n.Linear(middim[1],outdim)
  
  def forward(self,x):
    out=f.relu(self.fc1(x))
    out=f.relu(self.fc2(out))
    out=self.fc3(out)
    return out

In [0]:
model1=model(obsdim,[128,64],actiondim)

In [0]:
env=discreteonehotwrapper(gym.make("FrozenLake-v0"))

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
env.reset()

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [0]:
def iteratebatches(env,model,batchsize):
  batch=[]
  episodereward=0
  episodesteps=[]
  obs=env.reset()
  sm=n.Softmax(dim=1)
  while True:
    obs=torch.FloatTensor(obs).view(1,-1)
    action=model1.forward(obs)
    actionprobs=sm(action)
    #print(actionprobs.detach().numpy().shape)
    action=np.random.choice(actionprobs.shape[1],p=actionprobs.detach().numpy().reshape(-1,))
    nextobs,reward,isdone,_=env.step(action)
    episodereward+=reward
    episodesteps.append(episodestep(observation=obs,actions=action))
    if isdone:
      #print(episodereward)
      batch.append(episode(reward=episodereward,steps=episodesteps))
      episodesteps=[]
      episodereward=0
      obs=env.reset()
      if len(batch)==batchsize:
        yield batch
        batch=[]
    nextobs=obs

In [0]:
def filterbatches(batch,percentile):
  rewards=[x.reward for x in batch]
  return rewards

In [0]:
next(iteratebatches(env,model1,200))

[episode(reward=0.0, steps=[episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=1), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=3), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=3), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=2), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=3), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=3), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=0), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), actions=3), episodestep(observation=tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [0]:
#not possible since reward distribution is highly skewed and binary